## 解决并发请求使代码重复执行

业务逻辑:

- 调用接口获取 `foo`, `bar` 数据
- 检查是否存在可用更新，如果存在则更新本地文件
- 读取本地 `manifest.json` 文件返回数据

存在问题:

- 存在重复执行更新本地文件
- 会导致读取了正在写入的文件时程序报错

In [1]:
// 创建一个任务，在指定的毫秒数后完成
const delay = (ms = 1) => new Promise<void>((r) => setTimeout(r, ms));

In [2]:
const tasks = new Map<string, Promise<void>>();

async function visit(id: string): Promise<void> {
  let promise: Promise<void>;
  if (!tasks.has(id)) {
    promise = new Promise(async (r) => {
      console.log('check for new version...');
      console.log('new update available, writing file...');
      await delay(3000);
      console.log('update success!');
      r();
    });
    
    tasks.set(id, promise);
  } else {
    promise = tasks.get(id);
  }
    
  try {
    await promise;
  } finally {
    tasks.delete(id);
  }
}

In [3]:
// api/foo
async function getFoo(id: string): Promise<void> {
  await visit(id);
  console.log('read foo in manifest.json');
    
  // return foo
}

// api/bar
async function getBar(id: string): Promise<void> {
  await visit(id);
  console.log('read bar in manifest.json');
    
  // return bar
}

In [4]:
// 并发请求
getFoo('1').then();
getBar('1').then();

check for new version...
new update available, writing file...
Promise { <pending> }
update success!
read foo in manifest.json
read bar in manifest.json


从上面的输出可以看出来， `visit` 函数中的代码只被执行了一次